# 02. 농작물 가격 예측 모델

기상 데이터를 활용하여 6개 농작물의 가격을 예측합니다.

## 대상 농작물
- 사과 (apple)
- 양파 (onion)
- 배추 (baechu)
- 무 (radish)
- 감귤 (gyul)
- 복숭아 (peach)

## 사용 모델
- Linear Regression
- Ridge Regression (L2 정규화)
- Lasso Regression (L1 정규화)
- Polynomial Regression
- Random Forest Regressor

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from src import (
    load_crop_data,
    load_all_crops,
    CROPS,
    TARGET_COLUMN,
    CropPricePredictor,
    compare_models,
)

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

## 1. 데이터 로드 및 탐색

In [ ]:
# 전체 농작물 데이터 로드
crops_data = load_all_crops()

print('농작물별 데이터 shape:')
for crop, df in crops_data.items():
    print(f'  {crop} ({CROPS[crop]}): {df.shape}')

In [ ]:
# 사과 데이터 예시
apple_df = crops_data['apple']
apple_df.head()

In [ ]:
# 기초 통계량
apple_df.describe()

## 2. 데이터 시각화

In [ ]:
# 농작물별 가격 분포
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (crop, df) in enumerate(crops_data.items()):
    axes[i].hist(df[TARGET_COLUMN], bins=30, edgecolor='black', alpha=0.7)
    axes[i].set_title(f'{CROPS[crop]} 가격 분포')
    axes[i].set_xlabel(TARGET_COLUMN)
    axes[i].axvline(df[TARGET_COLUMN].mean(), color='red', linestyle='--', label=f'평균: {df[TARGET_COLUMN].mean():.0f}')
    axes[i].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 상관관계 히트맵 (사과 기준)
plt.figure(figsize=(10, 8))
sns.heatmap(apple_df.corr(), annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('사과 데이터 상관관계')
plt.tight_layout()
plt.show()

## 3. 단일 농작물 모델 학습 (사과)

In [ ]:
# 사과 가격 예측 모델
apple_predictor = CropPricePredictor(apple_df, 'apple')
apple_predictor.train_all_models(verbose=True)

In [ ]:
# 모델 성능 비교
apple_predictor.summary()

In [ ]:
# 최적 모델
best_name, best_result = apple_predictor.get_best_model()
print(f'최적 모델: {best_name}')
print(f'RMSE: {best_result.rmse:.2f}')
print(f'R²: {best_result.r2:.4f}')

In [ ]:
# Random Forest 피처 중요도
importance_df = apple_predictor.get_feature_importance()

plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df, x='importance', y='feature', palette='viridis')
plt.title('사과 가격 예측 - 피처 중요도 (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## 4. 전체 농작물 모델 비교

In [ ]:
# 모든 농작물에 대해 모델 학습 및 비교
comparison_df = compare_models(crops_data, verbose=True)

In [ ]:
# 결과 요약
comparison_df

In [ ]:
# RMSE vs 가격 표준편차 비교
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(comparison_df))
width = 0.35

ax.bar(x - width/2, comparison_df['RMSE'], width, label='RMSE', color='#e74c3c')
ax.bar(x + width/2, comparison_df['가격 표준편차'], width, label='가격 표준편차', color='#3498db')

ax.set_xlabel('농작물')
ax.set_ylabel('값')
ax.set_title('농작물별 RMSE vs 가격 표준편차')
ax.set_xticks(x)
ax.set_xticklabels([CROPS[c] for c in comparison_df['농작물']])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 결론

### 모델 성능 요약
- **양파, 감귤**: RMSE가 가격 표준편차보다 낮아 예측력 유의미
- **사과, 배추, 무, 복숭아**: RMSE가 표준편차와 비슷하거나 높아 추가 피처 필요

### 주요 피처 (Random Forest 기준)
- 거래량: 가격과 직접적인 상관관계
- 평균기온, 일교차: 계절성 반영
- 일강수량: 작황에 영향

### 한계점
- 전국 평균 기상 데이터 사용 → 지역별 세분화 필요
- 외부 요인(수입량, 정책 등) 미반영